# Texas Hold'em Monte Carlo Equity Simulator
**Author:** Will Tallard  
**Date:** December 11, 2025  
**Description:** This notebook simulates head-to-head poker equity for two starting hands using Monte Carlo simulations.

In [1]:
#Run this cell to install the hand evaluation library and random library used for this project. You will need it for the code to work.
!pip install phevaluator
#Run these import statements to get the required packages to run the code. 
import random
from phevaluator import Card, evaluate_cards
from tqdm.notebook import tqdm      # for progress bar

In [8]:

# --- DECK FUNCTIONS ---
#Creates a full deck of cards.
def create_deck():
    ranks = "23456789TJQKA"
    suits = "shdc"
    return [r+s for r in ranks for s in suits]
#Removes the 4 cards from the 2 players hands out of the deck.
def remove_used_cards(deck, hero, villain):
    d = deck.copy()
    for c in hero + villain:
        d.remove(c)
    return d
#Randomly draws 5 cards for the board
def deal_board(deck):
    return random.sample(deck, 5)

In [9]:
#Uses the imported module to score each 7 card hand (hole cards + board) combo to determine which one is better. A lower score is a stronger hand. 
def score_hand(cards):
    return evaluate_cards(*cards)


In [4]:
#This is the main function which runs the monte carlo simulation.
#Estimates equity for hero and villain in a heads-up Texas Hold'em scenario.

# --- SIMULATION FUNCTION ---
def simulate_equity(hero, villain, n=50000):
    hero_wins = 0
    villain_wins = 0
    ties = 0

    deck = create_deck()

    for i in tqdm(range(n), desc="Simulating"):
        remaining = remove_used_cards(deck, hero, villain)
        board = deal_board(remaining)

        h_score = score_hand(hero + board)
        v_score = score_hand(villain + board)

        if h_score < v_score:
            hero_wins += 1
        elif v_score < h_score:
            villain_wins += 1
        else:
            ties += 1

    return {
        "hero": hero_wins / n,
        "villain": villain_wins / n,
        "tie": ties / n
    }

In [5]:
# This converts a wide variety of string inputs into the correct format to work with the rest of the code. Returns a list of strings. 
def normalize_card(card):

    card = card.strip().lower()
    
    # Handle input without space (e.g., 'AhKh')
    cards = []
    i = 0
    while i < len(card):
        # Check for 10 (two-character rank)
        if card[i:i+2] == "10":
            rank = "T"
            suit = card[i+2]
            i += 3
        else:
            rank = card[i].upper()
            suit = card[i+1]
            i += 2
        
        if rank not in "23456789TJQKA" or suit not in "shdc":
            return None  # Invalid card
        cards.append(rank + suit.lower())
    
    return cards

In [6]:
# Makes sure the user inputted a valid notation for the cards
def get_hand(prompt):
    """Prompt user until a valid 2-card hand is entered"""
    while True:
        raw = input(prompt).replace(" ", "")  # remove spaces
        cards = normalize_card(raw)
        
        # Validate correct number of cards
        if cards is None or len(cards) != 2:
            print("❌ Invalid input. Example valid hands: Ah Kh, 10sTd, AhKh")
            continue
        
        # Check for duplicate cards in this hand
        if cards[0] == cards[1]:
            print("❌ Cannot use the same card twice in a hand. Try again.")
            continue
        
        return cards


In [17]:
# --- MAIN INTERACTIVE SIMULATION ---
#Run this cell and input the cards to get the win % for each hand. 
print("=== Poker Monte Carlo Simulator ===\n")
print("Input suits as: s (spades), h (hearts), d (diamonds), c (clubs) \n")
print("Example hands: Ah Kh, 10s Td, AhKh\n")



# Get hero hand
hero = get_hand("Enter HERO hand: ")

# Get villain hand; check no duplicates
while True:
    villain = get_hand("Enter VILLAIN hand: ")
    if set(hero) & set(villain):
        print("❌ Hero and Villain cannot share cards. Enter a different hand.\n")
    else:
        break

# Optional number of simulations
sim_raw = input("Number of simulations? (Press Enter for 50000): ").strip()
n = int(sim_raw) if sim_raw.isdigit() else 50000

print(f"\nRunning {n:,} simulations... Please wait.\n")

results = simulate_equity(hero, villain, n)

print("\n=== RESULTS ===")
print(f"Hero hand:     {hero}")
print(f"Villain hand:  {villain}\n")
print(f"Hero Win %:     {results['hero']*100:.2f}%")
print(f"Villain Win %:  {results['villain']*100:.2f}%")
print(f"Tie %:          {results['tie']*100:.2f}%")

=== Poker Monte Carlo Simulator ===

Input suits as: s (spades), h (hearts), d (diamonds), c (clubs) 

Example hands: Ah Kh, 10s Td, AhKh



Enter HERO hand:  as ad
Enter VILLAIN hand:  10hts
Number of simulations? (Press Enter for 50000):  



Running 50,000 simulations... Please wait.



Simulating:   0%|          | 0/50000 [00:00<?, ?it/s]


=== RESULTS ===
Hero hand:     ['As', 'Ad']
Villain hand:  ['Th', 'Ts']

Hero Win %:     80.61%
Villain Win %:  19.01%
Tie %:          0.38%
